<a href="https://colab.research.google.com/github/SaguPandya96/Research-Project-Fall-2020/blob/master/Proposed%20Work%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Hirarchical Attention Reset After Done
!pip install tensorflow==1.11.0

     |████████████████████████████████| 63.0MB 50kB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 573kB 53.5MB/s 
     |████████████████████████████████| 3.0MB 57.0MB/s 
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: google-auth 1.17.2 has requirement setuptools>=40.3.0, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 50.3.0
    Uninstalling setuptools-50.3.0:
      Successfully uninstalled setuptools-50.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
!pip install keras==2.1.5

     |████████████████████████████████| 337kB 2.7MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
import pandas as pd
import string
import random as rr
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import itertools
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
import re
from itertools import chain
from gensim.models import Word2Vec
from keras import backend as K
from keras.layers import Dense, Input
from keras.layers import GRU, Bidirectional, TimeDistributed, CuDNNLSTM, LSTM, Dropout, CuDNNGRU
from keras.models import Model
from keras.optimizers import  RMSprop
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from keras import backend as K
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarn

In [ ]:
#load data
DATA_PATH1 = '/content/drive/My Drive/Colab Notebooks/chromium.csv'
data1 = pd.read_csv(DATA_PATH1, sep='\t')
data_top1 = data1.head()
print("Chromium Data")
data_top1

Chromium Data


,title,description,type
0,Define a way to enforce performance constraint...,\nThere are cases where tests should fail if a...,Bug
1,Websites that don't work with Chrome Browser,\nChrome Version : 3.0.195.10\r\nURLs (i...,Compat
2,Redraw issue/truncation with confirmation item...,\nChromium 0.3.155.0 (Developer Build 3546)\r\...,Bug
3,Implement CanvasRenderingContext2D for access ...,\nApplications that do background image proces...,Feature
4,ath9k: panic in ath_tx_last_beacon + 0x29a,\nToT (3.0 kernel) panic'd on resume. Was ass...,Bug


In [ ]:
DATA_PATH2 = '/content/drive/My Drive/Colab Notebooks/linux_bugs_usage_ready.csv'
data2 = pd.read_csv(DATA_PATH2, sep='\t')
data_top2 = data2.head()
print("Linux Data")
data_top2

Linux Data


,title,message,importance,product,component
0,oops when using ide-cd with 2.5.45 and cdrecord,Please enter Exact Kernel version:2.5.45 Distr...,P2 normal,IO/Storage,IDE
1,NUMA-Q hangs during TSC initialization on boot.,Exact Kernel version: 2.5.46 Distribution: deb...,P2 normal,Platform Specific/Hardware,i386
2,Enabling shared pagetables causes KDE to wierd...,Exact Kernel version: 2.5.46-mm1 Distribution:...,P2 normal,Memory Management,Other
3,Dcache spirals out of control on 2.5.43-mm2,Exact Kernel version: 2.5.43-mm2 Distribution:...,P2 blocking,IO/Storage,Other
4,64GB highmem BUG(),Exact Kernel version: 2.5.40 Hardware Environm...,P2 normal,Memory Management,Other


In [ ]:
#merge title and message
def merge_title_and_message(data, message_col_name='message'):
  data['text'] = data['title'] + ' ' + data[message_col_name]
  data = data.drop(['title'], axis=1)
  data = data.drop([message_col_name], axis=1)
  return data
data1 = merge_title_and_message(data1, message_col_name='description')
data_top1 = data1.head()
print("Chromium Merge Title and Message Data")
data_top1

Chromium Merge Title and Message Data


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that don't work with Chrome Browser \...
2,Bug,Redraw issue/truncation with confirmation item...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k: panic in ath_tx_last_beacon + 0x29a \nT...


In [ ]:
data2 = merge_title_and_message(data2)
data_top2 = data2.head()
print("Linux Merge Title and Message Data")
data_top2

Linux Merge Title and Message Data


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using ide-cd with 2.5.45 and cdrecor...
1,P2 normal,Platform Specific/Hardware,i386,NUMA-Q hangs during TSC initialization on boot...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2.5.43-mm2 Ex...
4,P2 normal,Memory Management,Other,64GB highmem BUG() Exact Kernel version: 2.5.4...


In [ ]:
def strip_punctuations(data, column_name='text'):
  '''
  Strips punctuations from the end of each token.
  This uses suggestion from https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
  to accomplish this really fast.
  '''
  translator = str.maketrans('', '', string.punctuation)
  data['text'] = data['text'].map(lambda s : str(s).translate(translator))
  return data

In [ ]:
data1 = strip_punctuations(data1)
data_top1 = data1.head()
print("Chromium strip punctuations")
data_top1

Chromium strip punctuations


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that dont work with Chrome Browser nC...
2,Bug,Redraw issuetruncation with confirmation item ...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a nToT 30 ...


In [ ]:
data2 = strip_punctuations(data2)
data_top2 = data2.head()
print("Linux strip punctuations")
data_top2

Linux strip punctuations


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2543mm2 Exact...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


In [ ]:
def remove_linux_garbage(data):
  '''
  Linux data contains lots of garbage, e.g. memory addresses - 0000f800
  '''
  def is_garbage(w):
    return len(w) >= 7 and sum(c.isdigit() for c in w) >= 2

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if not is_garbage(w) else ' ', s.split())))
  return data
data = remove_linux_garbage(data2)
data_top2 = data2.head()
print("Remove linux garbage")
data_top2

Remove linux garbage


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on Exact Kerne...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


In [ ]:
def cast_to_lowercase(data):
  data['text'] = data['text'].map(lambda s : s.lower())
  return data
data1 = cast_to_lowercase(data1)
data2 = cast_to_lowercase(data2)
data_top1 = data1.head()
print("Chromium Lowercase")
data_top1

Chromium Lowercase


,type,text
0,Bug,define a way to enforce performance constraint...
1,Compat,websites that dont work with chrome browser nc...
2,Bug,redraw issuetruncation with confirmation item ...
3,Feature,implement canvasrenderingcontext2d for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a ntot 30 ...


In [ ]:
data_top2 = data2.head()
print("Linux Lowercase")
data_top2

Linux Lowercase


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord p...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs during tsc initialization on boot ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde to wierd...
3,P2 blocking,IO/Storage,Other,dcache spirals out of control on exact kerne...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [ ]:
def remove_stopwords(data):
  stop_words = stopwords.words('english')
  translator = str.maketrans('', '', string.punctuation)
  stop_words = set([w.translate(translator) for w in stop_words]) # Apostrophes were removed already

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if w not in stop_words else ' ', s.split())))
  return data
data1 = remove_stopwords(data1)
data2 = remove_stopwords(data2)
data_top1 = data1.head()
print("Chromium Stopword")
data_top1

Chromium Stopword


,type,text
0,Bug,define way enforce performance constraint ...
1,Compat,websites work chrome browser nchrome ver...
2,Bug,redraw issuetruncation confirmation item d...
3,Feature,implement canvasrenderingcontext2d access ...
4,Bug,ath9k panic athtxlastbeacon 0x29a ntot 30 ke...


In [ ]:
data_top2 = data2.head()
print("Linux Stopword")
data_top2

Linux Stopword


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please en...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ...
3,P2 blocking,IO/Storage,Other,dcache spirals control exact kernel vers...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [ ]:
def remove_rare_words(data, min_count=3):
  wc = {} # WordCount
  def proc_word(s):
    for w in set(s.split()):
      if w in wc:
        wc[w] += 1
      else:
        wc[w] = 1

  for index, row in data.iterrows():
    proc_word(row['text'])

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if wc[w] >= min_count else ' ', s.split())))
  return data
data1 = remove_rare_words(data1)
data2 = remove_rare_words(data2)
data_top1 = data1.head()
print("Chromium rare words")
data_top1

Chromium rare words


,type,text
0,Bug,define way enforce performance constraint spec...
1,Compat,websites work chrome browser nchrome version ...
2,Bug,redraw confirmation item download bar nchrom...
3,Feature,implement canvasrenderingcontext2d access work...
4,Bug,ath9k panic ntot 30 kernel resume associ...


In [ ]:
data_top2 = data2.head()
print("Linux rare words")
data_top2

Linux rare words


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please enter ex...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact kern...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ex...
3,P2 blocking,IO/Storage,Other,dcache control exact kernel version distribu...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [ ]:
translator = str.maketrans('', '', string.punctuation)
stop_words = stopwords.words('english')
stop_words = set([w.translate(translator) for w in stop_words])
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    string = string.strip().lower().translate(translator)
    return string
def remove_stopwords_from_sent(sent):
    res = []
    for word in sent:
        if word not in stop_words:
            res.append(word)
    return res
def map_sentence(sent):
    out = np.empty((max_sentence_len, embed_size_word2vec))
    for ind, word in enumerate(sent):
        if ind == max_sentence_len:
            break
        if word in vocabulary:
            out[ind, :] = wordvec_model.wv[word]
    return out
def map_doc(doc):
    out = np.empty((max_doc_len, max_sentence_len, embed_size_word2vec))
    for ind, sent in enumerate(doc):
        if ind == max_doc_len:
            break
        out[ind, :] = map_sentence(sent)
    return out
def build_sentences(X):
    X_sentences = []
    for doc in X:
        sentences = sent_tokenize(doc)
        cleaned = map(clean_str, sentences)
        tokenized = map(word_tokenize, cleaned)
        cleaned = map(remove_stopwords_from_sent, tokenized)
        X_sentences.append(list(cleaned))
    return X_sentences

In [ ]:
#Attension Layer
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
#RNN Class
from __future__ import absolute_import
import warnings
from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.engine import Layer
from keras.engine import InputSpec
from keras.legacy import interfaces
from keras.layers import RNN
from keras.layers.recurrent import _generate_dropout_mask, _generate_dropout_ones

class IndRNNCell(Layer):
    """Independently Recurrent Neural Networks Cell class.

    Derived from the paper [Independently Recurrent Neural Network (IndRNN): Building A Longer and Deeper RNN](https://arxiv.org/abs/1803.04831)
    Ref: [Tensorflow implementation](https://github.com/batzner/indrnn)

    # Arguments
        units: Positive integer, dimensionality of the output space.
        recurrent_clip_min: Can be one of None, -1 or float.
            If None, clipping of weights will not take place.
            If float, exact value will be used as clipping range
            If -1, will calculate the clip value for `relu` activation
        recurrent_clip_max: Can be one of None or float.
            If None, clipping of weights will not take place.
            If float, exact value will be used as clipping range
            If -1, will calculate the clip value for `relu` activation
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you pass None, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix,
            used for the linear transformation of the inputs
            (see [initializers](../initializers.md)).
        recurrent_initializer: Initializer for the `recurrent_kernel`
            weights matrix, used for the linear transformation of the
            recurrent state.
            Can be `None` or an available initializer. Defaults to `None`.
            If None, defaults to uniform initialization.
            If None, and recurrent_clip_min/max is not None, then
            it uses those clip values as for uniform initialization.
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        recurrent_regularizer: Regularizer function applied to
            the `recurrent_kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        recurrent_constraint: Constraint function applied to
            the `recurrent_kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
        dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the inputs.
        recurrent_dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the recurrent state.
        implementation: Implementation mode, must be 2.
            Mode 1 will structure its operations as a larger number of
            smaller dot products and additions, whereas mode 2 will
            batch them into fewer, larger operations. These modes will
            have different performance profiles on different hardware and
            for different applications.
    """

    def __init__(self, units,
                 recurrent_clip_min=-1,
                 recurrent_clip_max=-1,
                 activation='relu',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer=None,
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=2,
                 **kwargs):
        super(IndRNNCell, self).__init__(**kwargs)

        if implementation != 2:
            warnings.warn(
                "IndRNN only supports implementation 2 for the moment. Defaulting to implementation = 2")
            implementation = 2

        if recurrent_clip_min is None or recurrent_clip_max is None:
            recurrent_clip_min = None
            recurrent_clip_max = None

        self.units = units
        self.recurrent_clip_min = recurrent_clip_min
        self.recurrent_clip_max = recurrent_clip_max
        self.activation = activations.get(activation)
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer) \
                                     if recurrent_initializer is not None else None
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(recurrent_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(recurrent_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = min(1., max(0., dropout))
        self.recurrent_dropout = min(1., max(0., recurrent_dropout))
        self.implementation = implementation
        self.state_size = (self.units,)
        self._dropout_mask = None
        self._recurrent_masks = None

    def build(self, input_shape):
        input_dim = input_shape[-1]

        if self.recurrent_clip_min == -1 or self.recurrent_clip_max == -1:
            self.recurrent_clip_min = 0.0

            if hasattr(self, 'timesteps') and self.timesteps is not None:
                self.recurrent_clip_max = pow(2.0, 1. / self.timesteps)
            else:
                warnings.warn("IndRNNCell: Number of timesteps could not be determined. \n"
                              "Defaulting to max clipping range of 1.0. \n"
                              "If this model was trained using a specific timestep during training, "
                              "inference may be wrong due to this default setting.\n"
                              "Please ensure that you use the same number of timesteps during training "
                              "and evaluation")
                self.recurrent_clip_max = 1.0

        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      name='input_kernel',
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        if self.recurrent_initializer is None:
            if self.recurrent_clip_min is not None and self.recurrent_clip_max is not None:
                initialization_value = min(self.recurrent_clip_max, 1.0)
                self.recurrent_initializer = initializers.uniform(-initialization_value,
                                                                  initialization_value)
            else:
                self.recurrent_initializer = initializers.uniform(-1.0, 1.0)

        self.recurrent_kernel = self.add_weight(shape=(self.units,),
                                                name='recurrent_kernel',
                                                initializer=self.recurrent_initializer,
                                                regularizer=self.recurrent_regularizer,
                                                constraint=self.recurrent_constraint)

        if self.recurrent_clip_min is not None and self.recurrent_clip_max is not None:
            if abs(self.recurrent_clip_min):
                abs_recurrent_kernel = K.abs(self.recurrent_kernel)
                min_recurrent_kernel = K.maximum(abs_recurrent_kernel, abs(self.recurrent_clip_min))
                self.recurrent_kernel = K.sign(self.recurrent_kernel) * min_recurrent_kernel

            self.recurrent_kernel = K.clip(self.recurrent_kernel,
                                           self.recurrent_clip_min,
                                           self.recurrent_clip_max)

        if self.use_bias:
            bias_initializer = self.bias_initializer

            self.bias = self.add_weight(shape=(self.units,),
                                        name='bias',
                                        initializer=bias_initializer,
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True

    def call(self, inputs, states, training=None):
        if 0 < self.dropout < 1 and self._dropout_mask is None:
            self._dropout_mask = _generate_dropout_mask(
                _generate_dropout_ones(inputs, K.shape(inputs)[-1]),
                self.dropout,
                training=training,
                count=1)
        if (0 < self.recurrent_dropout < 1 and
                self._recurrent_masks is None):
            _recurrent_mask = _generate_dropout_mask(
                _generate_dropout_ones(inputs, self.units),
                self.recurrent_dropout,
                training=training,
                count=1)
            self._recurrent_masks = _recurrent_mask

        # dropout matrices for input units
        dp_mask = self._dropout_mask
        # dropout matrices for recurrent units
        rec_dp_masks = self._recurrent_masks

        h_tm1 = states[0]  # previous state

        if 0. < self.dropout < 1.:
            inputs *= dp_mask[0]

        if 0. < self.recurrent_dropout < 1.:
            h_tm1 *= rec_dp_masks[0]

        h = K.dot(inputs, self.kernel)
        h = h + (h_tm1 * self.recurrent_kernel)

        if self.use_bias:
            h = K.bias_add(h, self.bias)

        h = self.activation(h)

        if 0 < self.dropout + self.recurrent_dropout:
            if training is None:
                h._uses_learning_phase = True
        return h, [h]

    def get_config(self):
        config = {'units': self.units,
                  'recurrent_clip_min': self.recurrent_clip_min,
                  'recurrent_clip_max': self.recurrent_clip_max,
                  'activation': activations.serialize(self.activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer': initializers.serialize(self.recurrent_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                  'recurrent_regularizer': regularizers.serialize(self.recurrent_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'recurrent_constraint': constraints.serialize(self.recurrent_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'dropout': self.dropout,
                  'recurrent_dropout': self.recurrent_dropout,
                  'implementation': self.implementation}
        base_config = super(IndRNNCell, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class IndRNN(RNN):
    """Independently Recurrent Neural Networks Cell class.

    Derived from the paper [Independently Recurrent Neural Network (IndRNN): Building A Longer and Deeper RNN](https://arxiv.org/abs/1803.04831)
    Ref: [Tensorflow implementation](https://github.com/batzner/indrnn)

    # Arguments
        units: Positive integer, dimensionality of the output space.
        recurrent_clip_min: Can be one of None, -1 or float.
            If None, clipping of weights will not take place.
            If float, exact value will be used as clipping range
            If -1, computes the default clipping range for Relu activations
        recurrent_clip_max: Can be one of None, -1 or float.
            If None, clipping of weights will not take place.
            If float, exact value will be used as clipping range
            If -1, computes the default clipping range for Relu activations
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you pass None, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix,
            used for the linear transformation of the inputs.
            (see [initializers](../initializers.md)).
        recurrent_initializer: Initializer for the `recurrent_kernel`
            weights matrix,
            used for the linear transformation of the recurrent state.
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        unit_forget_bias: Boolean.
            If True, add 1 to the bias of the forget gate at initialization.
            Setting it to true will also force `bias_initializer="zeros"`.
            This is recommended in [Jozefowicz et al.](http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf)
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        recurrent_regularizer: Regularizer function applied to
            the `recurrent_kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        recurrent_constraint: Constraint function applied to
            the `recurrent_kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
        dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the inputs.
        recurrent_dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the recurrent state.
        implementation: Implementation mode, either 1 or 2.
            Mode 1 will structure its operations as a larger number of
            smaller dot products and additions, whereas mode 2 will
            batch them into fewer, larger operations. These modes will
            have different performance profiles on different hardware and
            for different applications.
        return_sequences: Boolean. Whether to return the last output.
            in the output sequence, or the full sequence.
        return_state: Boolean. Whether to return the last state
            in addition to the output.
        go_backwards: Boolean (default False).
            If True, process the input sequence backwards and return the
            reversed sequence.
        stateful: Boolean (default False). If True, the last state
            for each sample at index i in a batch will be used as initial
            state for the sample of index i in the following batch.
        unroll: Boolean (default False).
            If True, the network will be unrolled,
            else a symbolic loop will be used.
            Unrolling can speed-up a RNN,
            although it tends to be more memory-intensive.
            Unrolling is only suitable for short sequences.

    # References
        - [Learning to forget: Continual prediction with NestedLSTM](http://www.mitpressjournals.org/doi/pdf/10.1162/089976600300015015)
        - [Supervised sequence labeling with recurrent neural networks](http://www.cs.toronto.edu/~graves/preprint.pdf)
        - [A Theoretically Grounded Application of Dropout in Recurrent Neural Networks](http://arxiv.org/abs/1512.05287)
        - [Independently Recurrent Neural Network (IndRNN): Building A Longer and Deeper RNN](https://arxiv.org/abs/1803.04831)
    """

    @interfaces.legacy_recurrent_support
    def __init__(self, units,
                 recurrent_clip_min=-1,
                 recurrent_clip_max=-1,
                 activation='relu',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer=None,
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=2,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):
        if implementation == 0:
            warnings.warn('`implementation=0` has been deprecated, '
                          'and now defaults to `implementation=2`.'
                          'Please update your layer call.')
        if K.backend() == 'theano':
            warnings.warn(
                'RNN dropout is no longer supported with the Theano backend '
                'due to technical limitations. '
                'You can either set `dropout` and `recurrent_dropout` to 0, '
                'or use the TensorFlow backend.')
            dropout = 0.
            recurrent_dropout = 0.

        cell = IndRNNCell(units,
                          recurrent_clip_min=recurrent_clip_min,
                          recurrent_clip_max=recurrent_clip_max,
                          activation=activation,
                          use_bias=use_bias,
                          kernel_initializer=kernel_initializer,
                          recurrent_initializer=recurrent_initializer,
                          bias_initializer=bias_initializer,
                          kernel_regularizer=kernel_regularizer,
                          recurrent_regularizer=recurrent_regularizer,
                          bias_regularizer=bias_regularizer,
                          kernel_constraint=kernel_constraint,
                          recurrent_constraint=recurrent_constraint,
                          bias_constraint=bias_constraint,
                          dropout=dropout,
                          recurrent_dropout=recurrent_dropout,
                          implementation=implementation)
        super(IndRNN, self).__init__(cell,
                                     return_sequences=return_sequences,
                                     return_state=return_state,
                                     go_backwards=go_backwards,
                                     stateful=stateful,
                                     unroll=unroll,
                                     **kwargs)
        self.activity_regularizer = regularizers.get(activity_regularizer)

    def build(self, input_shape):
        timesteps = input_shape[1]

        if timesteps is None:
            warnings.warn("Number of timesteps was not provided. If this model is being used for training purposes, \n"
                          "it is recommended to provide a finite number of timesteps when defining the input shape, \n"
                          "so as to initialize the weights of the recurrent kernel properly and avoid exploding gradients.")

        self.cell.timesteps = timesteps

        super(IndRNN, self).build(input_shape)

    def call(self, inputs, mask=None, training=None, initial_state=None, constants=None):
        self.cell._dropout_mask = None
        self.cell._recurrent_masks = None
        return super(IndRNN, self).call(inputs,
                                        mask=mask,
                                        training=training,
                                        initial_state=initial_state,
                                        constants=constants)

    @property
    def units(self):
        return self.cell.units

    @property
    def recurrent_clip_min(self):
        return self.cell.recurrent_clip_min

    @property
    def recurrent_clip_max(self):
        return self.cell.recurrent_clip_max

    @property
    def activation(self):
        return self.cell.activation

    @property
    def use_bias(self):
        return self.cell.use_bias

    @property
    def kernel_initializer(self):
        return self.cell.kernel_initializer

    @property
    def recurrent_initializer(self):
        return self.cell.recurrent_initializer

    @property
    def bias_initializer(self):
        return self.cell.bias_initializer

    @property
    def kernel_regularizer(self):
        return self.cell.kernel_regularizer

    @property
    def recurrent_regularizer(self):
        return self.cell.recurrent_regularizer

    @property
    def bias_regularizer(self):
        return self.cell.bias_regularizer

    @property
    def kernel_constraint(self):
        return self.cell.kernel_constraint

    @property
    def recurrent_constraint(self):
        return self.cell.recurrent_constraint

    @property
    def bias_constraint(self):
        return self.cell.bias_constraint

    @property
    def dropout(self):
        return self.cell.dropout

    @property
    def recurrent_dropout(self):
        return self.cell.recurrent_dropout

    @property
    def implementation(self):
        return self.cell.implementation

    def get_config(self):
        config = {'units': self.units,
                  'recurrent_clip_min': self.recurrent_clip_min,
                  'recurrent_clip_max': self.recurrent_clip_max,
                  'activation': activations.serialize(self.activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer': initializers.serialize(self.recurrent_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                  'recurrent_regularizer': regularizers.serialize(self.recurrent_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'activity_regularizer': regularizers.serialize(self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'recurrent_constraint': constraints.serialize(self.recurrent_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'dropout': self.dropout,
                  'recurrent_dropout': self.recurrent_dropout,
                  'implementation': self.implementation}
        base_config = super(IndRNN, self).get_config()
        del base_config['cell']
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        if 'implementation' in config and config['implementation'] == 0:
            config['implementation'] = 2
        return cls(**config)

In [ ]:
def make_model(rnn_dim=64, dense_dim=50):
    def attention_block():
        def f(input):
            rnn2 = Bidirectional(IndRNN(rnn_dim, return_sequences=True))(input)
            drop2 = Dropout(0.75)(rnn2)
            dense = TimeDistributed(Dense(dense_dim))(drop2)
            drop3 = Dropout(0.5)(dense)
            att = AttentionWithContext()(drop3)
            return att
        return f
    with K.name_scope('sentence_enc'):
        sentence_input = Input(shape=(max_sentence_len, embed_size_word2vec))
        word_att = attention_block()(sentence_input)
        sentEncoder = Model(sentence_input, word_att)

    with K.name_scope('doc_enc'):
        doc_input = Input(shape=(max_doc_len, max_sentence_len, embed_size_word2vec))
        sent_enc = TimeDistributed(sentEncoder)(doc_input)
        doc_att = attention_block()(sent_enc)
        preds = Dense(y_trans.shape[-1], activation='softmax')(doc_att)

        model = Model(doc_input, preds)
        return model

In [ ]:
#Linux Process
def read_linux(feature):
    return data2['text'], data2[feature]

X, Y = read_linux('importance')
X_sentences = build_sentences(X)
list(map(print, X_sentences[2]))
# Word2vec parameters
min_word_frequency_word2vec = 3
embed_size_word2vec = 200
context_window_word2vec = 5
X_merged = list(map(lambda l: list(chain(*l)), X_sentences))
print(X_merged[13])
wordvec_model = Word2Vec(X_merged, min_count=min_word_frequency_word2vec,
                         size=embed_size_word2vec, window=context_window_word2vec)
max_doc_len = 5
max_sentence_len = 100
num = len(X_sentences)
vocabulary = wordvec_model.wv.vocab
print("Vocabulary", len(vocabulary))

['enabling', 'shared', 'pagetables', 'causes', 'kde', 'wierd', 'exact', 'kernel', 'version', 'distribution', 'redhat', '72', '73', 'hardware', 'environment', 'p4', 'pc', 'software', 'environment', 'kde', 'problem', 'description', 'enabling', 'shared', 'pagetables', 'causes', 'kde', 'wierd', 'steps', 'reproduce', 'start', 'kde']
['dri', 'unsupported', 'via', 'chipset', 'device', 'id', 'exact', 'kernel', 'version', 'kernel', 'command', 'line', 'ro', 'rootdevhdc1', 'agptryunsupported1', 'distribution', 'red', 'hat', 'rawhide', 'hardware', 'environment', 'gigabyte', 'ga', '7vax', 'httpwwwgigabytecomproducts7vaxhtm', 'northbridge', 'via', 'kt400', 'southbridge', 'via', '8235', 'latest', 'bios', 'mga', 'g400', 'boot', 'linux', 'agpgart', 'interface', 'v099', 'c', 'jeff', 'hartmann', 'agpgart', 'maximum', 'main', 'memory', 'use', 'agp', 'memory', '439m', 'agpgart', 'unsupported', 'via', 'chipset', 'device', 'id', 'might', 'want', 'try', 'agptryunsupported1', 'agpgart', 'supported', 'devices',

In [ ]:
x = np.memmap('filename1.myarray', dtype=np.float64, mode='w+',shape=(num, max_doc_len, max_sentence_len, embed_size_word2vec))
for ind, doc in enumerate(X_sentences):
    x[ind, :] = map_doc(doc)

In [ ]:
RNN = CuDNNGRU
enc = LabelEncoder()
yc = enc.fit_transform(Y)
oh = LabelBinarizer()
y_trans = oh.fit_transform(yc)
model = make_model(rnn_dim=64, dense_dim=64)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 5, 100, 200)       0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 5, 64)             38336     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 5, 128)            8448      
_________________________________________________________________
dropout_11 (Dropout)         (None, 5, 128)            0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 5, 64)             8256      
_________________________________________________________________
dropout_12 (Dropout)         (None, 5, 64)             0         
_________________________________________________________________
attention_with_context_6 (At (None, 64)                4224      
__________

In [ ]:
sz = len(y_trans)
x_train = x[int(0.1 * sz):int(0.95 * sz)]
x_test = np.concatenate((x[:int(0.1 * sz)], x[int(0.95 * sz):]))
y_train = y_trans[int(0.1 * sz):int(0.95 * sz)]
y_test = np.concatenate((y_trans[:int(0.1 * sz)], y_trans[int(0.95 * sz):]))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          nb_epoch=13, batch_size=16)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 13988 samples, validate on 2468 samples
Epoch 1/10
  272/13988 [..............................] - ETA: 7:46 - loss: nan - acc: 0.0110

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def report(x, y):
    labels = np.argmax(y, axis=-1)
    predicted = np.argmax(model.predict(x), axis=-1)
    print("Accuracy Linux", accuracy_score(labels, predicted))
    print("F1-Linux", f1_score(labels, predicted, average='weighted'))
print("Training")
report(x_train, y_train)
print("Testing")
report(x_test, y_test)

In [ ]:
#chrome Process
def read_chrome():
    data1['text'] = data1['text'].map(lambda s: str(s).replace('\\r', '').replace('\\n', '. '))
    return data1['text'], data1['type']
X, Y =read_chrome()
X_sentences = build_sentences(X)
list(map(print, X_sentences[2]))
# Word2vec parameters
min_word_frequency_word2vec = 3
embed_size_word2vec = 200
context_window_word2vec = 5
X_merged = list(map(lambda l: list(chain(*l)), X_sentences))
print(X_merged[13])
wordvec_model = Word2Vec(X_merged, min_count=min_word_frequency_word2vec,
                         size=embed_size_word2vec, window=context_window_word2vec)
max_doc_len = 5
max_sentence_len = 100
num = len(X_sentences)
vocabulary = wordvec_model.wv.vocab
print("Vocabulary", len(vocabulary))

['redraw', 'confirmation', 'item', 'download', 'bar', 'nchromium', '031550', 'developer', 'build', 'steps', 'reproduce', 'download', 'exe', 'file', 'tab', 'use', 'urlrn2', 'resize', 'chrome', 'window', 'making', 'narrower', 'current', 'item', 'disappearsrn3', 'switch', 'another', 'tab', 'resize', 'window', 'widthrn4', 'switch', 'back', 'tab', 'downloadsrnrnwhat', 'expected', 'output', 'rnrnthe', 'download', 'confirmation', 'item', 'properly', 'redrawn', 'download', 'barrnrnwhat', 'see', 'insteadrnrnthe', 'download', 'confirmation', 'item', 'truncated', 'resized', 'triggers', 'redraw', 'item']
['regression', 'aol', 'submit', 'button', 'aol', 'mail', 'registration', 'page', 'aligned', 'incorrectly', 'nchrome', 'version', '201690', 'official', 'build', '11238rnurls', 'applicable', 'browsers', 'testednadd', 'ok', 'fail', 'browsers', 'tested', 'issuen', 'firefox', '3', 'okrn', 'ie', '7', 'okrnrnwhat', 'steps', 'reproduce', 'problemn1', 'go', 'click', 'aol', 'mail', 'stuff', 'right', 'side',

In [ ]:
x = np.memmap('filename1.myarray', dtype=np.float64, mode='w+',shape=(num, max_doc_len, max_sentence_len, embed_size_word2vec))
for ind, doc in enumerate(X_sentences):
    x[ind, :] = map_doc(doc)

In [ ]:
RNN = CuDNNGRU
enc = LabelEncoder()
yc = enc.fit_transform(Y)
oh = LabelBinarizer()
y_trans = oh.fit_transform(yc)
model = make_model(rnn_dim=64, dense_dim=64)
model.summary()

In [ ]:
sz = len(y_trans)
x_train = x[int(0.1 * sz):int(0.95 * sz)]
x_test = np.concatenate((x[:int(0.1 * sz)], x[int(0.95 * sz):]))
y_train = y_trans[int(0.1 * sz):int(0.95 * sz)]
y_test = np.concatenate((y_trans[:int(0.1 * sz)], y_trans[int(0.95 * sz):]))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          nb_epoch=13, batch_size=16)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def report(x, y):
    labels = np.argmax(y, axis=-1)
    predicted = np.argmax(model.predict(x), axis=-1)
    print("Accuracy Crome", accuracy_score(labels, predicted))
    print("F1-Crome", f1_score(labels, predicted, average='weighted'))
print("Training")
report(x_train, y_train)
print("Testing")
report(x_test, y_test)